In [1]:
using CSV
using DataFrames
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/output_HHS_US_2021-01-09_one_column.csv"))

,Column1,hospital_name,collection_week,fips_code,hospitalization_percentage,hospitalization,target_date
,Int64,String,String,Int64,Float64?,Float64?,String
1,10,PRATTVILLE BAPTIST HOSPITAL,12/4/2020,1001,1.0,2.89626,12/15/2020
2,11,SOUTH BALDWIN REGIONAL MEDICAL CENTER,12/4/2020,1003,0.36965,3.69112,12/15/2020
3,12,THOMAS HOSPITAL,12/4/2020,1003,0.552529,5.51725,12/15/2020
4,13,NORTH BALDWIN INFIRMARY,12/4/2020,1003,0.077821,0.777078,12/15/2020
5,14,MEDICAL CENTER BARBOUR,12/4/2020,1005,1.0,0.565098,12/15/2020
6,15,BIBB MEDICAL CENTER,12/4/2020,1007,1.0,1.14839,12/15/2020
7,16,ST VINCENTS BLOUNT,12/4/2020,1009,1.0,4.16832,12/15/2020
8,17,BULLOCK COUNTY HOSPITAL,12/4/2020,1011,1.0,0.195106,12/15/2020
9,18,REGIONAL MEDICAL CENTER OF CENTRAL ALABAMA,12/4/2020,1013,1.0,0.760204,12/15/2020


In [4]:
data = select(rawdata,
    :hospital_name => ByRow(titlecase) => :hospital,
    :target_date => ByRow(x -> Date(x, dateformat"mm/dd/yy")) => :date,
    :hospitalization => ByRow(x -> x*0.3) => :admitted_moderate_icu,
    :hospitalization => ByRow(x -> x*0.7) => :admitted_moderate_acute,
    :hospitalization => :admitted_moderate_allbeds,
)

,hospital,date,admitted_moderate_icu,admitted_moderate_acute,admitted_moderate_allbeds
,String,Date,Float64?,Float64?,Float64?
1,Prattville Baptist Hospital,2020-12-15,0.868877,2.02738,2.89626
2,South Baldwin Regional Medical Center,2020-12-15,1.10734,2.58378,3.69112
3,Thomas Hospital,2020-12-15,1.65518,3.86208,5.51725
4,North Baldwin Infirmary,2020-12-15,0.233123,0.543955,0.777078
5,Medical Center Barbour,2020-12-15,0.16953,0.395569,0.565098
6,Bibb Medical Center,2020-12-15,0.344518,0.803875,1.14839
7,St Vincents Blount,2020-12-15,1.2505,2.91782,4.16832
8,Bullock County Hospital,2020-12-15,0.0585317,0.136574,0.195106
9,Regional Medical Center Of Central Alabama,2020-12-15,0.228061,0.532143,0.760204


In [5]:
data |> CSV.write("../data/hhs_forecast_2020_12_24.csv")

"../data/hhs_forecast_2020_12_24.csv"